Importing the dependencies

In [1]:
# https://drive.google.com/file/d/1uzbhec5TW_OjFr4UUZkoMm0rpyvYdhZw/view

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

Data collection and preprocessing

In [2]:
mail_data = pd.read_csv('mail_data.csv')

In [3]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# Replace all the null values with a null string
mail_data =mail_data.where((pd.notnull(mail_data)), '')

In [5]:
mail_data.isnull().sum()

Category    0
Message     0
dtype: int64

In [6]:
mail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
mail_data.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [8]:
mail_data['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
mail_data.shape

(5572, 2)

LabelEncoding
- Spam mail : 0
- Ham mail : 1

In [ ]:
mail_data.replace({"Category" : { "spam" : 0, "ham" : 1}}, inplace= True)

#  OR

# mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
# mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

C:\Users\Mahdi\AppData\Local\Temp\ipykernel_11224\2822670718.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mail_data.replace({"Category" : { "spam" : 0, "ham" : 1}}, inplace= True)


Splitting the data into Text and labels

In [12]:
X = mail_data['Message']
Y = mail_data['Category']

In [14]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [15]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: int64


Train test Split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, random_state= 3)

In [19]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4457,), (1115,), (4457,), (1115,))

Feature Extraction

In [23]:
# Transform the text data to feature vector that can be used as an input to the Logistic Regression
feature_extraction = TfidfVectorizer(min_df= 1, stop_words= 'english', lowercase= True)

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

In [25]:
print(X_test_features)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7687 stored elements and shape (1115, 7431)>
  Coords	Values
  (0, 1)	0.2381316303003606
  (0, 9)	0.2852706805264544
  (0, 14)	0.26797874471323896
  (0, 20)	0.30668032384591537
  (0, 306)	0.23975986557206702
  (0, 405)	0.2381316303003606
  (0, 1041)	0.28016206931555726
  (0, 1082)	0.2451068436245027
  (0, 1361)	0.25132445289897426
  (0, 1405)	0.3176863938914351
  (0, 1549)	0.2646498848307188
  (0, 4386)	0.18353336340308998
  (0, 5213)	0.1988547357502182
  (0, 5373)	0.2365698724638063
  (0, 6920)	0.20571591693537986
  (0, 7271)	0.1940327008179069
  (1, 3491)	0.496093956101028
  (1, 4418)	0.3457696891316818
  (1, 4729)	0.22965776503163893
  (1, 6214)	0.3621564482127515
  (1, 6507)	0.26731535902873493
  (1, 6588)	0.3298937975962767
  (1, 6732)	0.42473488678029325
  (1, 7368)	0.29957800964520975
  (2, 201)	0.2824102268489399
  :	:
  (1110, 6591)	0.7327660015422193
  (1111, 2440)	0.4137350055985486
  (1111, 3227)	0.44384935772735

### Training the model

In [26]:
model = LogisticRegression()
model.fit(X_train_features, Y_train)

LogisticRegression()

Evaluating the trained model

In [27]:
# Prediction on trained data
X_train_prediction = model.predict(X_train_features)
trained_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [28]:
print("Accuracy Score:", trained_data_accuracy)

Accuracy Score: 0.9676912721561588


In [29]:
# Prediction on test data
X_test_prediction = model.predict(X_test_features)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [30]:
print("Accuracy Score:", test_data_accuracy)

Accuracy Score: 0.9668161434977578


Predictive System

In [33]:
input = ["Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input)

# Makint prediction
prediction = model.predict(input_data_features)
print(prediction[0])

if (prediction[0] == 0):
    print("SPAM mail")
else:
    print("Ham mail")

1
Ham mail
